In [36]:
import pandas as pd
import numpy as np
import os
import json

In [9]:
d = "./result"

dirs_to_analyze = [os.path.join(d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o)) and "\\experiment" in os.path.join(d, o)]


In [42]:
for curdir in dirs_to_analyze:
    print(curdir + "/accuracy.json")
    try:
        accuracy = pd.read_json(curdir + "/accuracy.json").transpose()
    except Exception as e:
        print(e)
        
    try:
        time_summary = json.load(open(curdir + "/summary.json"))
    except Exception as e:
        print(e)
    

./result\experiment1_0.001_128_10_1G_1_0.01/accuracy.json
./result\experiment1_0.001_128_10_1G_2_0.01/accuracy.json
./result\experiment1_0.001_128_10_2G_1_0.01/accuracy.json
./result\experiment1_0.001_128_5_1G_1_0.01/accuracy.json
./result\experiment1_0.001_128_5_1G_2_0.01/accuracy.json
./result\experiment1_0.001_128_5_2G_1_0.01/accuracy.json
./result\experiment1_0.001_128_5_2G_2_0.01/accuracy.json
./result\experiment1_0.001_32_10_1G_1_0.01/accuracy.json
./result\experiment1_0.001_32_10_1G_2_0.01/accuracy.json
./result\experiment1_0.001_32_10_2G_1_0.01/accuracy.json
./result\experiment1_0.001_32_10_2G_2_0.01/accuracy.json
./result\experiment1_0.001_32_5_1G_1_0.01/accuracy.json
./result\experiment1_0.001_32_5_1G_2_0.01/accuracy.json
./result\experiment1_0.001_32_5_2G_1_0.01/accuracy.json
./result\experiment1_0.001_32_5_2G_2_0.01/accuracy.json
./result\experiment1_0.01_128_10_1G_1_0.01/accuracy.json
./result\experiment1_0.01_128_10_1G_2_0.01/accuracy.json
./result\experiment1_0.01_128_10